<a href="https://colab.research.google.com/github/Rtniewi/kcwiertniewicz-IDS/blob/main/GenAI/HW5/Assignment5_FINAL4LSTMLAYERSipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

//***Katrina Cwiertniewicz
//*** CSC 330
//11/22/2024
//Assignment 5: Text Generation Using LSTM on Project Gutenberg Training Data
####The purpose of this assignment is to develop an LSTM model that generates text. The goal is to produce coherent and stylistically relevant text based on prompts.

In [1]:
import numpy as np
import json
import re
import string

import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, losses

## 0. Parameters <a name="parameters"></a>

In [2]:
VOCAB_SIZE = 20049
MAX_LEN = 500
EMBEDDING_DIM = 100
N_UNITS_LAYER_1 = 64
N_UNITS_LAYER_2 = 128
N_UNITS_LAYER_3 = 256
N_UNITS_LAYER_4 = 256
VALIDATION_SPLIT = 0.2
SEED = 42
LOAD_MODEL = False
BATCH_SIZE = 32
EPOCHS = 25

## 1. Load the data <a name="load"></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%pwd

'/content'

In [5]:
import requests
import os


# List of files for additional texts (e.g., different Edgar Allan Poe Works)
file_paths = [
  os.path.join('/content/drive/MyDrive/text/The_Tell_Tale_Heart.txt'),                        # The Tell Tale Heart
  os.path.join('/content/drive/MyDrive/text/The_Cask_of_Amontillado.txt'),                    # The Cask of Amontillado
  os.path.join('/content/drive/MyDrive/text/The_Raven.txt'),                                  # The Raven
  os.path.join('/content/drive/MyDrive/text/The_Masque.txt'),                                 # The Masque of the Red Death
  os.path.join('/content/drive/MyDrive/text/Annabel_Lee.txt'),                                # Annabel Lee
  os.path.join('/content/drive/MyDrive/text/Lenore.txt'),                                     # Lenore
  os.path.join('/content/drive/MyDrive/text/The_Bells.txt'),                                  # The Bells
  os.path.join('/content/drive/MyDrive/text/The_Black_Cat.txt'),                              # The Black Cat
  os.path.join('/content/drive/MyDrive/text/The_Fall_of_the_House_of_Usher.txt'),             # The Fall of the House of Usher
  os.path.join('/content/drive/MyDrive/text/The_Oval_Portrait.txt'),                          # The Oval Portrait
  os.path.join('/content/drive/MyDrive/text/The_Pit_and_the_Pendulum.txt'),                   # The Pit and the Pendulum
  os.path.join('/content/drive/MyDrive/text/The_Premature_Burial.txt'),                       # The Premature Burial
  os.path.join('/content/drive/MyDrive/text/The_Narrative_of_Arthur_Gordon.txt'),             # The Narrative of Arthur Gordon Pym of Nantucket
  os.path.join('/content/drive/MyDrive/text/Al_Aaraaf.txt'),                                  # Al Aaraaf
  os.path.join('/content/drive/MyDrive/text/Alone.txt'),                                      # Alone
  os.path.join('/content/drive/MyDrive/text/Eureka.txt'),                                     # Eureka
  os.path.join('/content/drive/MyDrive/text/The_Facts_in_the_Case_of_M._Valdemar.txt'),       # The Facts in the Case of M.Valdemar
  os.path.join('/content/drive/MyDrive/text/A_Descent_into_the_Maelstrom.txt'),               # A Descent into the Maelstrom
  os.path.join('/content/drive/MyDrive/text/William_Wilson.txt'),                             # William Wilson
  os.path.join('/content/drive/MyDrive/text/Berenice.txt'),                                   # Berenice
  os.path.join('/content/drive/MyDrive/text/The_Gold_Bug.txt'),                               # The Gold Bug
  os.path.join('/content/drive/MyDrive/text/The_Murders_of_Rue_Morgue.txt'),                  # The Murders in the Rue Morgue
  os.path.join('/content/drive/MyDrive/text/The_Mystery_of_Marie_Roget.txt'),                 # The Mystery of Marie Roget
  os.path.join('/content/drive/MyDrive/text/The_Purloined_Letter.txt'),                       # The Purloined Letter
  os.path.join('/content/drive/MyDrive/text/Von_Kempelen_and _his_Discovery.txt'),            # Von Kempelen and His Discovery
  os.path.join('/content/drive/MyDrive/text/Island_of_the_Fay.txt'),                          # Island of the Fay
  os.path.join('/content/drive/MyDrive/text/Mesmeric_Revelation.txt'),                        # Mesemeric Revelation
  os.path.join('/content/drive/MyDrive/text/Silence_A_Fable.txt'),                            # Silence a Fable
  os.path.join('/content/drive/MyDrive/text/MS._Found_in_a_Bottle.txt'),                      # MS. Found in a Bottle
  os.path.join('/content/drive/MyDrive/text/The_Thousand_and_Second_Tale_of_Scherezade.txt'), # The Thousand and Second Tale of Scherezade
  os.path.join('/content/drive/MyDrive/text/The_Unparalleled_Adventure.txt'),                 # The Unparalleled Adventure of One Hans Pfaall
  os.path.join('/content/drive/MyDrive/text/The_Assignation.txt'),                            # The Assignation
  os.path.join('/content/drive/MyDrive/text/The_Imp.txt'),                                    # The Imp of the Perverse
  os.path.join('/content/drive/MyDrive/text/The_Domain_of_Arnheim.txt'),                      # The Domain of Arnheim
  os.path.join('/content/drive/MyDrive/text/The_Assignation.txt'),                            # Landor's Cottage
  os.path.join('/content/drive/MyDrive/text/Morella.txt'),                                    # Morella
  os.path.join('/content/drive/MyDrive/text/Ligeia.txt'),                                     # Ligeia
  os.path.join('/content/drive/MyDrive/text/King_Pest.txt'),                                  # King Pest
  os.path.join('/content/drive/MyDrive/text/A_Tale_of_the_Ragged_Mountains.txt'),             # A Tale of the Ragged Mountains
  os.path.join('/content/drive/MyDrive/text/The_Spectacles.txt'),                             # The Spectacles
  os.path.join('/content/drive/MyDrive/text/Philosophy_of_Furniture.txt'),                    # The Philosophy of Furniture
  os.path.join('/content/drive/MyDrive/text/The_Devil_in_Belfry.txt'),                        # The Devil in the Belfry
  os.path.join('/content/drive/MyDrive/text/Bon_Bon.txt'),                                    # Bon-Bon
  os.path.join('/content/drive/MyDrive/text/Some_Words_with_a_Mummy.txt')                     # Some Words with a Mummy


]

# Initialize an empty string to hold all text
all_text = ""

# Download each text file and append to all_text
for file_path in file_paths:
  with open(file_path, 'r') as file:
    content = file.read()
    text = content
    all_text += text + "\n\n"  # Separate texts by newlines

# Save combined text to a single file
  with open('/content/combined_poe.txt', "w", encoding="utf-8") as file:
    file.write(all_text)


In [6]:
# Count the words of text
with open('/content/combined_poe.txt', "r", encoding="utf-8") as file:
  file_content = file.read()
  words = file_content.split()
  n_words = len(words)
print(f"{n_words} words loaded")

326500 words loaded


In [7]:
# Example Sentence of First Ten Words
example_sentence = words[:10]
print(f"Example Sentence: {example_sentence}")

Example Sentence: ['True!—nervous—very,', 'very', 'dreadfully', 'nervous', 'I', 'had', 'been', 'and', 'am;', 'but']


## 2. Tokenise the data

In [8]:
# Pad the punctuation, to treat them as separate 'words'
def pad_punctuation(s):
    s = re.sub(f"([{string.punctuation}])", r" \1 ", s)
    s = re.sub(" +", " ", s)
    return s


with open("combined_poe.txt", "r", encoding="utf-8") as file:
    text_data = [pad_punctuation(line) for line in file]

In [9]:
example_date = text_data[30]
print(example_date)

Ha ! —would a madman have been so wise as this ? And then , when my



In [10]:
# Convert to a Tensorflow Dataset
text_ds = (
    tf.data.Dataset.from_tensor_slices(text_data)
    .batch(BATCH_SIZE)
    .shuffle(1000)
)

In [11]:
# Create a vectorisation layer
vectorize_layer = layers.TextVectorization(
    standardize="lower",
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    output_sequence_length=MAX_LEN + 1,
)

In [12]:
# Adapt the layer to the training set
vectorize_layer.adapt(text_ds)
vocab = vectorize_layer.get_vocabulary()

# Length of Vocabulary
print(f"Length of Vocabulary: {len(vocab)}")

Length of Vocabulary: 20049


In [13]:
# Display some token:word mappings
for i, word in enumerate(vocab[10:30]):
    print(f"{i}: {word}")

0: -
1: i
2: that
3: _
4: it
5: was
6: as
7: with
8: which
9: at
10: my
11: is
12: had
13: ;
14: we
15: this
16: for
17: by
18: not
19: be


In [14]:
# Display the same example converted to ints
example_tokenised = vectorize_layer(text_data)
print(example_tokenised.numpy())

[[  326    47 11929 ...     0     0     0]
 [   33   329    74 ...     0     0     0]
 [13630 18134 10712 ...     0     0     0]
 ...
 [    0     0     0 ...     0     0     0]
 [    0     0     0 ...     0     0     0]
 [    0     0     0 ...     0     0     0]]


## 3. Create the Training Set

In [15]:
# Create the training set of text and the same text shifted by one word
def prepare_inputs(text):
    text = tf.expand_dims(text, -1)
    tokenized_sentences = vectorize_layer(text)
    x = tokenized_sentences[:, :-1]
    y = tokenized_sentences[:, 1:]
    return x, y


train_ds = text_ds.map(prepare_inputs)

## 4. Build the LSTM <a name="build"></a>

In [16]:
inputs = layers.Input(shape=(None,), dtype="int32")
x = layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM)(inputs)
x = layers.LSTM(N_UNITS_LAYER_1, return_sequences=True)(x)
x = layers.LSTM(N_UNITS_LAYER_2, return_sequences=True)(x)
x = layers.LSTM(N_UNITS_LAYER_3, return_sequences=True)(x)
x = layers.LSTM(N_UNITS_LAYER_4, return_sequences=True)(x)
outputs = layers.Dense(VOCAB_SIZE, activation="softmax")(x)
lstm = models.Model(inputs, outputs)
lstm.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, None)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, None, 100)           │       2,004,900 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, None, 64)            │          42,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, None, 128)           │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, None, 256)           │         394,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, None, 256)           │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, None, 20049)         │       5,152,593 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,218,101 (31.35 MB)

 Trainable params: 8,218,101 (31.35 MB)

 Non-trainable params: 0 (0.00 B)

## 5. Train the LSTM <a name="train"></a>

In [17]:
loss_fn = losses.SparseCategoricalCrossentropy()
lstm.compile("adam", loss_fn)

In [18]:
# Create a TextGenerator checkpoint
class TextGenerator(callbacks.Callback):
    def __init__(self, index_to_word, top_k=10):
        self.index_to_word = index_to_word
        self.word_to_index = {
            word: index for index, word in enumerate(index_to_word)
        }  # <1>

    def sample_from(self, probs, temperature):  # <2>
        probs = probs ** (1 / temperature)
        probs = probs / np.sum(probs)
        return np.random.choice(len(probs), p=probs), probs

    def generate(self, start_prompt, max_tokens, temperature):
        start_tokens = [
            self.word_to_index.get(x, 1) for x in start_prompt.split()
        ]  # <3>
        sample_token = None
        info = []
        while len(start_tokens) < max_tokens and sample_token != 0: # <4>
            x = np.array([start_tokens])
            y = self.model.predict(x, verbose=0)  # <5>
            sample_token, probs = self.sample_from(y[0][-1], temperature)  # <6>
            info.append({"prompt": start_prompt, "word_probs": probs})
            start_tokens.append(sample_token)  # <7>
            start_prompt = start_prompt + " " + self.index_to_word[sample_token]
        print(f"\ngenerated text:\n{start_prompt}\n")
        return info

    def on_epoch_end(self, epoch, logs=None):
        self.generate("Text:", max_tokens=1000, temperature=1.0)

In [19]:
# Tokenize starting prompt

text_generator = TextGenerator(vocab)

In [20]:
lstm.fit(
    train_ds,
    epochs=EPOCHS,
    callbacks=[text_generator],
)

Epoch 1/25
1035/1035 ━━━━━━━━━━━━━━━━━━━━ 0s 346ms/step - loss: 0.7873
generated text:
Text: unfastened i ; difficulty upon to , ! confirmation such at the had 

1035/1035 ━━━━━━━━━━━━━━━━━━━━ 365s 347ms/step - loss: 0.7868
Epoch 2/25
1035/1035 ━━━━━━━━━━━━━━━━━━━━ 0s 347ms/step - loss: 0.1429
generated text:
Text: for in ” outward cried the wreaths of the thing from in fell the 

1035/1035 ━━━━━━━━━━━━━━━━━━━━ 360s 348ms/step - loss: 0.1429
Epoch 3/25
1035/1035 ━━━━━━━━━━━━━━━━━━━━ 0s 349ms/step - loss: 0.1363
generated text:
Text: - rather from its liveliness state and my small chamber of first 

1035/1035 ━━━━━━━━━━━━━━━━━━━━ 362s 350ms/step - loss: 0.1363
Epoch 4/25
1035/1035 ━━━━━━━━━━━━━━━━━━━━ 0s 348ms/step - loss: 0.1282
generated text:
Text: the wife and the whirl had if held we unable acquainted . 

1035/1035 ━━━━━━━━━━━━━━━━━━━━ 361s 349ms/step - loss: 0.1282
Epoch 5/25
1035/1035 ━━━━━━━━━━━━━━━━━━━━ 0s 348ms/step - loss: 0.1274
generated text:
Text: us , , which i found at 

## 6. Generate text using the LSTM

In [21]:
def print_probs(info, vocab, top_k=5):
    for i in info:
        print(f"\nPROMPT: {i['prompt']}")
        word_probs = i["word_probs"]
        p_sorted = np.sort(word_probs)[::-1][:top_k]
        i_sorted = np.argsort(word_probs)[::-1][:top_k]
        for p, i in zip(p_sorted, i_sorted):
            print(f"{vocab[i]}:   \t{np.round(100*p,2)}%")
        print("--------\n")

In [45]:
# Prompt 1: From "The Raven"    Temperature=0.1
info = text_generator.generate(
    """Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and curious volume of forgotten lore,
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door.
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more.""", max_tokens=1000, temperature=0.1
)


generated text:
Once upon a midnight dreary, while I pondered, weak and weary, 
      Over many a quaint and curious volume of forgotten lore, 
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door. 
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more. , i sat upon the springs — i had been buried with a violent cry , and , upon the closing around the shadows — the habits of the land were a white and terrible — that i gazed upon the curtains of arnheim and more terrible changes of berenice , and in the most fashion of the forest , and a speck of suffocation , i turned myself to the wall , and , in the first place , to loom it to the contemplation of the library - - a thing being , of the most fashion of the library’s heavens , i told them to shudder to ponder and horror , and , lastly the terror , and a record of the most terrible and terrible , and invariably —

In [53]:
print_probs(info, vocab)

Streaming output truncated to the last 5000 lines.
--------


PROMPT: Once upon a midnight dreary, while I pondered, weak and weary, 
      Over many a quaint and curious volume of forgotten lore, 
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door. 
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more. — told myself in the antechamber too unintelligible every individuality ? an energy and air under the musings - lilies with her ropes . a few ghastly most melancholy . her heart was curtains upon her bed , and looked down into her hands , and turned the maniac of a fire not — rich , ghastly , suffocation , and slipped away with the changing and form of a mental spectrum of the dormitories . but by the air of my imagination — and it was proved deeply since it , as i pressed them away , and i fell up alone , when i had loved the imagination of her malady . he fir

In [48]:
# Prompt 1: From "The Raven"    Temperature=0.5
info = text_generator.generate(
    """Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and curious volume of forgotten lore,
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door.
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more.""", max_tokens=1000, temperature=0.5
)


generated text:
Once upon a midnight dreary, while I pondered, weak and weary, 
      Over many a quaint and curious volume of forgotten lore, 
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door. 
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more. , with a wild smile , a condition of her smile , but , in the meantime of my heart , i beheld them , and i shudder . i had died upon the springs of the night , and , in the meantime , having fallen up the corpse , and , as i was told , i found them , and i thought the very step , and looked away by the contemplation of the library — but the wild and ghastly shriek , i sat upon the curtains of the flower and desultory me ! oh ! berenice ! — 



In [52]:
print_probs(info, vocab)

Streaming output truncated to the last 5000 lines.
--------


PROMPT: Once upon a midnight dreary, while I pondered, weak and weary, 
      Over many a quaint and curious volume of forgotten lore, 
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door. 
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more. — told myself in the antechamber too unintelligible every individuality ? an energy and air under the musings - lilies with her ropes . a few ghastly most melancholy . her heart was curtains upon her bed , and looked down into her hands , and turned the maniac of a fire not — rich , ghastly , suffocation , and slipped away with the changing and form of a mental spectrum of the dormitories . but by the air of my imagination — and it was proved deeply since it , as i pressed them away , and i fell up alone , when i had loved the imagination of her malady . he fir

In [51]:
# Prompt 1: From "The Raven"    Temperature=1.0
info = text_generator.generate(
    """Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and curious volume of forgotten lore,
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door.
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more.""", max_tokens=1000, temperature=1.0
)


generated text:
Once upon a midnight dreary, while I pondered, weak and weary, 
      Over many a quaint and curious volume of forgotten lore, 
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door. 
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more. — told myself in the antechamber too unintelligible every individuality ? an energy and air under the musings - lilies with her ropes . a few ghastly most melancholy . her heart was curtains upon her bed , and looked down into her hands , and turned the maniac of a fire not — rich , ghastly , suffocation , and slipped away with the changing and form of a mental spectrum of the dormitories . but by the air of my imagination — and it was proved deeply since it , as i pressed them away , and i fell up alone , when i had loved the imagination of her malady . he first beheld myself in regard departed wrought : and the

In [54]:
print_probs(info, vocab)

Streaming output truncated to the last 5000 lines.
--------


PROMPT: Once upon a midnight dreary, while I pondered, weak and weary, 
      Over many a quaint and curious volume of forgotten lore, 
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door. 
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more. — told myself in the antechamber too unintelligible every individuality ? an energy and air under the musings - lilies with her ropes . a few ghastly most melancholy . her heart was curtains upon her bed , and looked down into her hands , and turned the maniac of a fire not — rich , ghastly , suffocation , and slipped away with the changing and form of a mental spectrum of the dormitories . but by the air of my imagination — and it was proved deeply since it , as i pressed them away , and i fell up alone , when i had loved the imagination of her malady . he fir

In [59]:
# Prompt 2: From "The Tell Tale Heart"  Temperature=0.1
info = text_generator.generate(
    """And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart.
     It increased my fury, as the beating of a drum stimulates the soldier into courage.""", max_tokens=1000, temperature=0.1
)


generated text:
And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart. 
     It increased my fury, as the beating of a drum stimulates the soldier into courage. and the chamber . 



In [60]:
print_probs(info, vocab)


PROMPT: And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart. 
     It increased my fury, as the beating of a drum stimulates the soldier into courage.
and:   	100.0%
.:   	0.0%
in:   	0.0%
,:   	0.0%
—:   	0.0%
--------


PROMPT: And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart. 
     It increased my fury, as the beating of a drum stimulates the soldier into courage. and
the:   	100.0%
their:   	0.0%
my:   	0.0%
her:   	0.0%
a:   	0.0%
--------


PROMPT: And have I not told you that what you mistake for madness is but over

In [61]:
# Prompt 2: From "The Tell Tale Heart"  Temperature=0.5
info = text_generator.generate(
    """And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart.
     It increased my fury, as the beating of a drum stimulates the soldier into courage.""", max_tokens=1000, temperature=0.5
)


generated text:
And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart. 
     It increased my fury, as the beating of a drum stimulates the soldier into courage. and the chamber . the coffin like my hand , and i beheld myself to the curtains of my monomania . i knew it ? i told them to know , however , and i shudder . but , then , a few more and gentle sound , and , upon the chamber of dim paintings , joy . it was a mystery , and secured with a gentle investigation , i sat , fell with the curtains of mortality eyes , and lustreless . 



In [62]:
print_probs(info, vocab)


PROMPT: And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart. 
     It increased my fury, as the beating of a drum stimulates the soldier into courage.
and:   	89.78%
.:   	8.91%
in:   	1.01%
,:   	0.16%
—:   	0.06%
--------


PROMPT: And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart. 
     It increased my fury, as the beating of a drum stimulates the soldier into courage. and
the:   	98.32%
their:   	0.83%
my:   	0.39%
her:   	0.27%
a:   	0.1%
--------


PROMPT: And have I not told you that what you mistake for madness is b

In [66]:
# Prompt 2: From "The Tell Tale Heart"  Temperature=1.0
info = text_generator.generate(
    """And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart.
     It increased my fury, as the beating of a drum stimulates the soldier into courage.""", max_tokens=1000, temperature=1.0
)


generated text:
And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart. 
     It increased my fury, as the beating of a drum stimulates the soldier into courage. and their lips — and why as we had been buried over the attitude of them seemed so told , peters death as i raised my walls , low open , and smoke . with living awe before the day was the teeth were hidden more over more or breathing . it springs , and called , but great days , but the impossibility of being more away into my eyes . alas — then she had not been understood rife by alarm for some in common arch or infirmity , madame mused . hers iron feared , and meditations with the thin eyes of the heart ; in their scrutiny , i raised my acquaintance in troubled my knees , succeeded with a grave beh

In [67]:
print_probs(info, vocab)


PROMPT: And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart. 
     It increased my fury, as the beating of a drum stimulates the soldier into courage.
and:   	63.22%
.:   	19.92%
in:   	6.71%
,:   	2.63%
—:   	1.67%
--------


PROMPT: And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart. 
     It increased my fury, as the beating of a drum stimulates the soldier into courage. and
the:   	73.49%
their:   	6.74%
my:   	4.63%
her:   	3.84%
a:   	2.36%
--------


PROMPT: And have I not told you that what you mistake for madness is

In [71]:
# Prompt 3: From the Mystery of Marie Roget   Temperature=0.1
info = text_generator.generate(
    """There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a character that, as
    mere coincidences, the intellect has been unable to receive them. """, max_tokens=1000, temperature=0.1
)


generated text:
There are few persons, even among the calmest thinkers, who have not 
    occasionally been startled into a vague yet thrilling half-credence in the 
    supernatural, by coincidences of so seemingly marvellous a character that, as 
    mere coincidences, the intellect has been unable to receive them.  ; but it was not the memory of the fearful condition that i fell upon the spot in the early morning , and the fiercer and the shriek . i now not be told . i struggled to decypher my life , but i was born . i had not heard the triple winds of the grave . 



In [72]:
print_probs(info, vocab)


PROMPT: There are few persons, even among the calmest thinkers, who have not 
    occasionally been startled into a vague yet thrilling half-credence in the 
    supernatural, by coincidences of so seemingly marvellous a character that, as 
    mere coincidences, the intellect has been unable to receive them. 
;:   	99.94%
,:   	0.06%
!:   	0.0%
?:   	0.0%
.:   	0.0%
--------


PROMPT: There are few persons, even among the calmest thinkers, who have not 
    occasionally been startled into a vague yet thrilling half-credence in the 
    supernatural, by coincidences of so seemingly marvellous a character that, as 
    mere coincidences, the intellect has been unable to receive them.  ;
but:   	100.0%
and:   	0.0%
for:   	0.0%
a:   	0.0%
i:   	0.0%
--------


PROMPT: There are few persons, even among the calmest thinkers, who have not 
    occasionally been startled into a vague yet thrilling half-credence in the 
    supernatural, by coincidences of so seemingly marvellous a character

In [119]:
# Prompt 3: From the Mystery of Marie Roget   Temperature=0.5
info = text_generator.generate(
    """There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a character that, as
    mere coincidences, the intellect has been unable to receive them. """, max_tokens=1000, temperature=0.5
)


generated text:
There are few persons, even among the calmest thinkers, who have not 
    occasionally been startled into a vague yet thrilling half-credence in the 
    supernatural, by coincidences of so seemingly marvellous a character that, as 
    mere coincidences, the intellect has been unable to receive them.  ; but it was not a matter in sorrow , and upon the spirit of imagination and erudition issuing to the evidence . 



In [120]:
print_probs(info, vocab)


PROMPT: There are few persons, even among the calmest thinkers, who have not 
    occasionally been startled into a vague yet thrilling half-credence in the 
    supernatural, by coincidences of so seemingly marvellous a character that, as 
    mere coincidences, the intellect has been unable to receive them. 
;:   	67.31%
,:   	15.23%
!:   	7.46%
?:   	7.09%
.:   	1.95%
--------


PROMPT: There are few persons, even among the calmest thinkers, who have not 
    occasionally been startled into a vague yet thrilling half-credence in the 
    supernatural, by coincidences of so seemingly marvellous a character that, as 
    mere coincidences, the intellect has been unable to receive them.  ;
but:   	78.59%
and:   	9.63%
for:   	3.21%
a:   	2.26%
i:   	1.96%
--------


PROMPT: There are few persons, even among the calmest thinkers, who have not 
    occasionally been startled into a vague yet thrilling half-credence in the 
    supernatural, by coincidences of so seemingly marvellous a c

In [117]:
# Prompt 3: From the Mystery of Marie Roget   Temperature=1.0
info = text_generator.generate(
    """There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a character that, as
    mere coincidences, the intellect has been unable to receive them. """, max_tokens=1000, temperature=1.0
)


generated text:
There are few persons, even among the calmest thinkers, who have not 
    occasionally been startled into a vague yet thrilling half-credence in the 
    supernatural, by coincidences of so seemingly marvellous a character that, as 
    mere coincidences, the intellect has been unable to receive them.  ? i staggered involuntarily with my chair , and fell on . 



In [118]:
print_probs(info, vocab)


PROMPT: There are few persons, even among the calmest thinkers, who have not 
    occasionally been startled into a vague yet thrilling half-credence in the 
    supernatural, by coincidences of so seemingly marvellous a character that, as 
    mere coincidences, the intellect has been unable to receive them. 
;:   	36.11%
,:   	17.18%
!:   	12.02%
?:   	11.72%
.:   	6.15%
--------


PROMPT: There are few persons, even among the calmest thinkers, who have not 
    occasionally been startled into a vague yet thrilling half-credence in the 
    supernatural, by coincidences of so seemingly marvellous a character that, as 
    mere coincidences, the intellect has been unable to receive them.  ?
i:   	14.02%
:   	13.72%
”:   	9.33%
but:   	7.9%
for:   	5.64%
--------


PROMPT: There are few persons, even among the calmest thinkers, who have not 
    occasionally been startled into a vague yet thrilling half-credence in the 
    supernatural, by coincidences of so seemingly marvellous a ch